# 测试

In [1]:
from PIL import Image
import numpy as np
import mxnet as mx
from tvm.contrib.download import download_testdata
import tvm
from tvm import relay
from tvm.contrib import graph_executor

## 导入前端模型

In [2]:
model_name = 'mobilenet1.0'
gluon_model = mx.gluon.model_zoo.vision.get_model(model_name, pretrained=True)

## 下载、预处理和加载测试图像

当涉及到预期的张量形状、格式和数据类型时，每个模型都很特别。出于这个原因，大多数模型需要一些预处理和后处理，以确保输入是有效的，并解释输出。

下载图像数据，然后将其转换成 numpy 数组，作为模型的输入。

In [3]:
img_url = "https://s3.amazonaws.com/model-server/inputs/kitten.jpg"
img_path = download_testdata(img_url, "imagenet_cat.png", module="data")

# resize 到 224x224
with Image.open(img_path) as im:
    resized_image = im.resize((224, 224))

# 转换为 float32
img_data = np.asarray(resized_image).astype("float32")

# 输入图像是在 HWC 布局，而 MXNet 期望 CHW 输入
img_data = np.transpose(img_data, (2, 0, 1))

# 根据 ImageNet 输入规范进行 Normalize
imagenet_mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
imagenet_stddev = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
norm_img_data = (img_data / 255 - imagenet_mean) / imagenet_stddev

# 添加批处理维度，设置数据为 4 维 输入：NCHW
img_data = np.expand_dims(norm_img_data, axis=0)

## 用 Relay 编译模型

下一步是编译 ResNet 模型。使用 `from_mxnet` 导入器将模型导入到 relay。然后，将模型与标准优化一起构建成 TVM 库。最后，从该库中创建 TVM graph 运行时模块。

In [4]:
input_name = "data"
target = "llvm"

shape_dict = {input_name: img_data.shape}
mod, params = relay.frontend.from_mxnet(gluon_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

/media/workspace/anaconda3/envs/mx/lib/python3.10/site-packages/tvm/driver/build_module.py:263: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


获取推理结果：

In [5]:
dtype = "float32"
module.set_input(input_name, img_data)
module.run()
output_shape = (1, 1000)
tvm_output = module.get_output(0,
                               tvm.nd.empty(output_shape)).numpy()

In [6]:
import timeit

timing_number = 10
timing_repeat = 10
unoptimized = (
    np.array(timeit.Timer(lambda: module.run()).repeat(repeat=timing_repeat, number=timing_number))
    * 1000
    / timing_number
)
unoptimized = {
    "mean": np.mean(unoptimized),
    "median": np.median(unoptimized),
    "std": np.std(unoptimized),
}

print(unoptimized)

{'mean': 8.031886797398329, 'median': 7.966969162225723, 'std': 0.1568694376261838}
